In [1]:
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'JioKisan.settings')

import django
django.setup()

In [2]:
from trucks.models import Driver,Delivery
import numpy as np

In [3]:
# Drivers which are not hired
# Consignments which are pending
drivers = Driver.objects.all().filter(hired=False)
consignments = Delivery.objects.all().filter(status="PENDING") # ADD FURTHER FILTERS LIKE PICKUP date if required

num_drivers = drivers.count()
num_consignments = consignments.count()

# A square matrix representing driverLocation(all) , conignmentLocationPickup, conignmentLocationDrop 
# addreses (all consignment one by one) in a row
# This matrix will contain road distance obtained by DistanceMatrixAPI

In [14]:


def initializeMappingList(num_drivers, num_consignments, drivers, consignments):

    # A list of dicts map indices of driver/pickup/drop  
    mappingList = []
    for i in range(num_drivers+2*num_consignments):
        if(i<num_drivers):
            entryType = 'Driver'
            pk = drivers[i].pk
            address = str(drivers[i].currentPositionLatitude)+','+str(drivers[i].currentPositionLongitude) 
            weight = drivers[i].truckCapacity
        else:
            k=i-num_drivers
            consignmentEntry_index = int(k/2)

            if(k%2==0):
                entryType = 'Pickup Location'
                pk = consignments[consignmentEntry_index].pk
                address = str(consignments[consignmentEntry_index].pickupLocationLatitude) + ',' + str(consignments[consignmentEntry_index].pickupLocationLongitude)

            else:
                entryType = 'Drop Location'
                pk = consignments[consignmentEntry_index].pk
                address = str(consignments[consignmentEntry_index].dropLocationLatitude) + ',' + str(consignments[consignmentEntry_index].dropLocationLongitude)

            weight = consignments[consignmentEntry_index].weight
            
        infoDict = {
            'entryType' : entryType,
            'pk' : pk,
            'address' : address,
            'weight' : weight
        } 

        mappingList.append(infoDict)
        
    return mappingList


In [30]:
def fillDistanceMatrix(originList, destinationList):
    
    distanceMatrix = np.random.randint(1,15,size=(len(destinationList),len(originList)))
    
    maxValue = np.max(distanceMatrix)
    
    
    for i in originList:
        if i['entryType'] == 'Driver':
            for j in destinationList:
                if (j['entryType'] == 'Pickup Location') and (j['weight']>i['weight']):
                    distanceMatrix[destinationList.index(j)][originList.index(i)] = maxValue + 1
                    
                    
    
    return distanceMatrix


In [64]:
def driverDeliveryAssignment(mappingList, num_drivers, num_consignments):
    # This List will Contain indexes of location in mappingList and in order for driver has to go 
    locations = []
    assignedConsignments_num = 0
    # A list dictionaries like info dict containing only pickup location parts of assigned consignments
    assignedConsignments = []  
    
    originList = [ mappingList[i] for i in range(num_drivers) ]
    destinationList = [ mappingList[i] for i in range(num_drivers, len(mappingList), 2) ]
 
    
    for i in originList:
        if i['entryType']!='Driver':
            print('Something Wrong in Origin List')
            
    
    for i in destinationList:
        if i['entryType']!='Pickup Location':
            print('Something Wrong in Destination List')
    
    
       
    # Origins are along column & destinations are along rows
#     distanceMatrix = np.zeros(shape=(len(destinationList),len(originList)))
    
    #Compute Distances 
    distanceMatrix = fillDistanceMatrix(originList, destinationList) 
    

    firstPickup_index = np.where(distanceMatrix == np.min(distanceMatrix))[0][0] 
    firstDriver_index =  np.where(distanceMatrix == np.min(distanceMatrix))[1][0]
    
    firstPickup = destinationList[firstPickup_index]
    
    firstDriver = originList[firstDriver_index]
    

    #    let myDriver be a driver object from django model while 
    #   firstDriver is dictionary containing details about myDriver including pk    
    
    #  Make this driver hired.
   
    assignedConsignments_num = 1
    locations.append(firstPickup)
    
    
    loadedWeight = firstPickup['weight']
    
    print('Got the first pickup loaded weight is ', loadedWeight, 'truck weight is ', firstDriver['weight'])
    
    originList = []
    originList.append(firstPickup)
    destinationList.remove(firstPickup)
    destinationList.append(mappingList[mappingList.index(firstPickup)+1])
    assignedConsignments.append(firstPickup)
    
    
    
    
    while(len(destinationList)>0):
    
        # We have to filter left consignments according to truck capacity or further filters can be added here
        # An example of another filter may me to remove consignments with different crop
        removeIndices = []
        for i in destinationList:
            if i['entryType'] == 'Pickup Location':   # A drop location might still be in destination list
                if loadedWeight + i['weight'] > firstDriver['weight']:
                    removeIndices.append(i)      # Remove Delivery with more weight
        for i in removeIndices:
            destinationList.remove(i)
                    
        
        
        
        distanceMatrix = fillDistanceMatrix(originList, destinationList)
        
        
        nextLocation_index = firstPickup_index = np.where(distanceMatrix == np.min(distanceMatrix))[0][0]
    
        nextLocation = destinationList[nextLocation_index]
        
        destinationList.remove(nextLocation)
        
        
        
        if(nextLocation['entryType'] == 'Pickup Location'):
            assignedConsignments_num = assignedConsignments_num + 1
            if(assignedConsignments_num<6):
                loadedWeight = loadedWeight + nextLocation['weight']
                assignedConsignments.append(nextLocation)
                locations.append(nextLocation)
                destinationList.append(mappingList[mappingList.index(nextLocation)+1])
        
        else:
            locations.append(nextLocation)
            
    
    
    return firstDriver, locations, assignedConsignments
    
    
    
    

















In [81]:
mappingList = initializeMappingList(num_drivers, num_consignments, drivers, consignments)
assignedDriver , pathOfDriver, assignedConsignments = driverDeliveryAssignment(mappingList, num_drivers, num_consignments)

Got the first pickup loaded weight is  421 truck weight is  1212


In [78]:
assignedDriver

{'entryType': 'Driver', 'pk': 5, 'address': '12.1277,78.15794', 'weight': 1310}

In [79]:
pathOfDriver

[{'entryType': 'Pickup Location',
  'pk': 10,
  'address': '23.1959,86.51499',
  'weight': 421},
 {'entryType': 'Pickup Location',
  'pk': 9,
  'address': '22.10194,85.37752',
  'weight': 789},
 {'entryType': 'Drop Location',
  'pk': 9,
  'address': '23.49668,86.68363',
  'weight': 789},
 {'entryType': 'Drop Location',
  'pk': 10,
  'address': '22.75218,72.68533',
  'weight': 421}]

In [80]:
assignedConsignments

[{'entryType': 'Pickup Location',
  'pk': 10,
  'address': '23.1959,86.51499',
  'weight': 421},
 {'entryType': 'Pickup Location',
  'pk': 9,
  'address': '22.10194,85.37752',
  'weight': 789}]

# Testing

In [38]:
testDriver = Driver.objects.all().filter(hired=False)
testconsignments = Delivery.objects.all().filter(status="PENDING")
testconsignments.count()
testDriver[0].currentPositionLongitude

78.14797

In [40]:
testDriver[3].hired

False

In [40]:
c = 4
a = [2, 3, 4, 5, 7, 5, 9, 3, 31, 73, 89, 2]
b = [a[2*i] for i in range(int(c/2),int(len(a)/2))]
b

[7, 9, 31, 89]

In [15]:
a.index(4)

2

In [46]:
c = 4
a = [2, 3, 4, 5, 7, 5, 9, 3, 31, 73, 89, 2]
b = [ a[i] for i in range(c,len(a),2) ] 

7
9
31
89


In [32]:
b.pop(1)

9

In [57]:
b = [7, 9, 31, 8, 89]
for i in b:
    print('i = ', i)
    if(i>10):
        print("Removing ",i)
        b.remove(i)
b

i =  7
i =  9
i =  31
Removing  31
i =  89
Removing  89


[7, 9, 8]

In [49]:
b

[7, 9, 89]

In [43]:
b

[7, 9, 89]